In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import Counter
import plotly.express as px
from sklearn.cluster import DBSCAN
import xgboost as xgb
import lightgbm as lgb


In [2]:
online_retail_cleaned_no_cancellations=pd.read_csv(r"C:\Desktop\UK Based Retail Analytics\online_retail_cleaned_no_cancellations.csv")

In [ ]:
online_retail_cleaned_no_cancellations=pd.read_csv("cleaned_online_retail_data_no_cancellations_no_outliers_for_clustering.csv")


In [ ]:
online_retail_cleaned_no_cancellations.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice(£),CustomerID,Country,Sales(£),InvoiceMonth,InvoiceDay,InvoiceHour,InvoiceWeekday,is_canceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,12,1,8,2,False
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,1,8,2,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,12,1,8,2,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,1,8,2,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,1,8,2,False


## Step 1: Feature Engineering

### 1.3 Lagged Averages for Price and Quantity Trends


In [ ]:
""""Detecting price and quantity trends, to compute the lagged averages for UnitPrice(£) and Quantity per product."""

In [3]:
# Lagged average for Quantity and UnitPrice for each product (StockCode)
online_retail_cleaned_no_cancellations['Quantity_Lagged_Avg'] = online_retail_cleaned_no_cancellations.groupby('StockCode')['Quantity'].transform(lambda x: x.shift(1).rolling(window=3).mean())
online_retail_cleaned_no_cancellations['UnitPrice_Lagged_Avg'] = online_retail_cleaned_no_cancellations.groupby('StockCode')['UnitPrice(£)'].transform(lambda x: x.shift(1).rolling(window=3).mean())

print(online_retail_cleaned_no_cancellations[['StockCode', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']].head())


  StockCode  Quantity_Lagged_Avg  UnitPrice_Lagged_Avg
0    85123A                  NaN                   NaN
1     71053                  NaN                   NaN
2    84406B                  NaN                   NaN
3    84029G                  NaN                   NaN
4    84029E                  NaN                   NaN


In [4]:
print(online_retail_cleaned_no_cancellations[['StockCode', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])


       StockCode  Quantity_Lagged_Avg  UnitPrice_Lagged_Avg
0         85123A                  NaN                   NaN
1          71053                  NaN                   NaN
2         84406B                  NaN                   NaN
3         84029G                  NaN                   NaN
4         84029E                  NaN                   NaN
...          ...                  ...                   ...
397919     22613             5.666667              0.850000
397920     22899             3.666667              2.100000
397921     23254             2.666667              4.150000
397922     23255             6.000000              4.150000
397923     22138            24.666667              4.483333

[397924 rows x 3 columns]


In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


# Step 1: Convert 'InvoiceDate' to datetime
online_retail_cleaned_no_cancellations['InvoiceDate'] = pd.to_datetime(online_retail_cleaned_no_cancellations['InvoiceDate'])

# Step 2: Extract time-related features
online_retail_cleaned_no_cancellations['Hour'] = online_retail_cleaned_no_cancellations['InvoiceDate'].dt.hour
online_retail_cleaned_no_cancellations['Day'] = online_retail_cleaned_no_cancellations['InvoiceDate'].dt.dayofweek  # 0 = Monday, 6 = Sunday
online_retail_cleaned_no_cancellations['Month'] = online_retail_cleaned_no_cancellations['InvoiceDate'].dt.month

# Step 3: Encode 'StockCode'(Label Encoding or One-Hot Encoding)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# Encode 'StockCode'
online_retail_cleaned_no_cancellations['StockCode_Encoded'] = label_encoder.fit_transform(online_retail_cleaned_no_cancellations['StockCode'])



## Price Prediction Model


### Data Preparation

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

# Step 1: Handle Outliers
# Detect outliers using Z-score method
z_scores = np.abs(zscore(online_retail_cleaned_no_cancellations[['Quantity', 'UnitPrice(£)', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']]))
outliers = (z_scores > 3).any(axis=1)
cleaned_data = online_retail_cleaned_no_cancellations[~outliers]

# Step 2: Handle Missing Values
# Impute missing values using SimpleImputer with median strategy
imputer = SimpleImputer(strategy='median')
cleaned_data_imputed = cleaned_data.copy()

# Apply imputation to the relevant features
cleaned_data_imputed[['Quantity', 'UnitPrice(£)', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']] = imputer.fit_transform(
    cleaned_data[['Quantity', 'UnitPrice(£)', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])

# Step 3: Feature Scaling
# Use StandardScaler to scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(cleaned_data_imputed[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])

# Replace the original features with scaled ones
cleaned_data_imputed[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']] = scaled_features

# Step 4: Create Interaction Features
cleaned_data_imputed['Quantity_UnitPrice_Lagged_Avg'] = cleaned_data_imputed['Quantity'] * cleaned_data_imputed['UnitPrice_Lagged_Avg']

# Step 5: Encode 'StockCode'
label_encoder = LabelEncoder()
cleaned_data_imputed['StockCode_Encoded'] = label_encoder.fit_transform(cleaned_data_imputed['StockCode'])

# Prepare the input features (X) and target variable (y)
X = cleaned_data_imputed[['StockCode_Encoded', 'Quantity', 'Hour', 'Day', 'Month', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg', 'Quantity_UnitPrice_Lagged_Avg']]
y = cleaned_data_imputed['UnitPrice(£)']

# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### Training Model 

In [7]:
# Step 7: Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


# Step 8: Train the Linear Regression model (for comparison)
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)



LinearRegression()

### Model Evaluation

In [8]:
# Step 9: Evaluate the Random Forest model
y_pred_rf = rf_model.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)


print(f"Random Forest with feature engineering:")
print(f"Mean Absolute Error (MAE): {mae_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")
print(f"R-squared (R²): {r2_rf:.4f}")


Random Forest with feature engineering:
Mean Absolute Error (MAE): 0.1220
Root Mean Squared Error (RMSE): 0.7932
R-squared (R²): 0.9386


In [9]:

# Step 10: Evaluate the Linear Regression model
y_pred_lr = lr_model.predict(X_test)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print(f"\nLinear Regression with feature engineering:")
print(f"Mean Absolute Error (MAE): {mae_lr:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_lr:.4f}")
print(f"R-squared (R²): {r2_lr:.4f}")




Linear Regression with feature engineering:
Mean Absolute Error (MAE): 1.8951
Root Mean Squared Error (RMSE): 3.0188
R-squared (R²): 0.1106


### Predict Unit Price Function

In [10]:
# Step 11: Predict Unit Price Function
def predict_unit_price(stock_code, invoice_weekday, invoice_hour, invoice_month, historical_data, rf_model, scaler):
    """
    Predicts the unit price of a stock dynamically based on customer inputs.

    Args:
        stock_code (int): The encoded stock code.
        invoice_weekday (int): Day of the week (0 = Monday, ..., 6 = Sunday).
        invoice_hour (int): Hour of the day in 24-hour format.
        invoice_month (int): Month of the year (1 = January, ..., 12 = December).
        historical_data (DataFrame): Historical data for lagged averages and scaling.
        rf_model (RandomForestRegressor): Trained model.
        scaler (StandardScaler): Fitted scaler for scaling numeric features.
        
    Returns:
        float: Predicted unit price (£).
    """
    # Filter data for the given stock code
    stock_data = historical_data[historical_data['StockCode_Encoded'] == stock_code]
    if stock_data.empty:
        raise ValueError(f"No historical data found for StockCode {stock_code}")

    # Calculate lagged averages
    quantity_lagged_avg = stock_data['Quantity_Lagged_Avg'].mean()
    unitprice_lagged_avg = stock_data['UnitPrice_Lagged_Avg'].mean()
    quantity_unitprice_interaction = quantity_lagged_avg * unitprice_lagged_avg

    # Create input feature
    input_features = pd.DataFrame({
        'StockCode_Encoded': [stock_code],
        'Quantity': [quantity_lagged_avg],
        'Hour': [invoice_hour],
        'Day': [invoice_weekday],
        'Month': [invoice_month],
        'Quantity_Lagged_Avg': [quantity_lagged_avg],
        'UnitPrice_Lagged_Avg': [unitprice_lagged_avg],
        'Quantity_UnitPrice_Lagged_Avg': [quantity_unitprice_interaction]
    })

    # Scale numeric features
    scaled_features = scaler.transform(input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])
    input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']] = scaled_features

    # Predict using the model
    predicted_price = rf_model.predict(input_features)
    return predicted_price[0]



In [ ]:
# Example Prediction
stock_code_encoded = label_encoder.transform(['85123A'])[0]  #'85131D' with your desired stock code
predicted_price = predict_unit_price(
    stock_code=stock_code_encoded,
    invoice_weekday=3,  # Wednesday
    invoice_hour=17,    # 5 PM
    invoice_month=12,   # December
    historical_data=cleaned_data_imputed,
    rf_model=rf_model,
    scaler=scaler
)
print(f"\nPredicted Unit Price for StockCode ' 85123A': £{predicted_price:.2f}")




Predicted Unit Price for StockCode ' 85123A': £0.08


### Saving the trained Model

In [15]:
import joblib

# Save the trained RandomForest model, scaler, and label encoder
joblib.dump(rf_model, 'random_forest_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']

## Build the Streamlit App:


In [14]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np
# Load the trained model, scaler, and label encoder
#rf_model = joblib.load('random_forest_model.pkl')
#scaler = joblib.load('scaler.pkl')
#label_encoder = joblib.load('label_encoder.pkl')

# Title of the web app
st.title('Unit Price Prediction for Stock')

# Create input fields for the user
stock_code = st.text_input('Enter Stock Code (e.g., "85123A")', '85123A')
invoice_weekday = st.selectbox('Select Invoice Weekday', ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
invoice_hour = st.slider('Select Invoice Hour', 0, 23, 15)  # Default to 3 PM (15:00)
invoice_month = st.slider('Select Invoice Month', 1, 12, 11)  # Default to November

# Mapping weekdays to integers (0=Monday, 6=Sunday)
weekday_mapping = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3,
    'Friday': 4, 'Saturday': 5, 'Sunday': 6
}
invoice_weekday_int = weekday_mapping[invoice_weekday]

# When the user presses the button to predict
if st.button('Predict Unit Price'):
    # Encode the stock code
    stock_code_encoded = label_encoder.transform([stock_code])[0]

    # Create the input features (replace lagged values with actual user input if available)
    input_features = pd.DataFrame({
        'StockCode_Encoded': [stock_code_encoded],
        'Quantity': [0],  # You can replace this with actual input from the user
        'InvoiceHour': [invoice_hour],
        'InvoiceWeekday': [invoice_weekday_int],
        'InvoiceMonth': [invoice_month],
        'Quantity_Lagged_Avg': [0],  # Replace with actual input if available
        'UnitPrice_Lagged_Avg': [0],  # Replace with actual input if available
        'Quantity_UnitPrice_Lagged_Avg': [0]  # Replace with actual input if available
    })

    # Scale the features
    scaled_features = scaler.transform(input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])
    input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']] = scaled_features

    # Make prediction
    predicted_price = rf_model.predict(input_features)

    # Display the predicted price to the user
    st.write(f"Predicted Unit Price for StockCode {stock_code} on {invoice_weekday} at {invoice_hour}:00 is: £{predicted_price[0]:.2f}")



2024-12-07 21:37:59.932 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 21:38:01.997 
  command:

    streamlit run c:\Users\godwi\Downloads\Anconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-07 21:38:01.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 21:38:01.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 21:38:02.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 21:38:02.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 21:38:02.003 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 21:38:02.004 Session state d

In [ ]:
import streamlit as st
import pandas as pd
import joblib
import os

# Function to load model, scaler, label encoder, and dataset
@st.cache_resource
def load_model_and_data():
    # Check if all required files exist
    if not os.path.exists('random_forest_model.pkl') or not os.path.exists('scaler.pkl') or not os.path.exists('label_encoder.pkl') or not os.path.exists('online_retail_cleaned_no_cancellations.csv'):
        st.error("Required files (model/scaler/encoder/data) are missing! Please check the directory.")
        st.stop()
    

    # Load the trained model, scaler, label encoder, and data
    model = joblib.load('random_forest_model.pkl')
    scaler = joblib.load('scaler.pkl')
    label_encoder = joblib.load('label_encoder.pkl')
    data = pd.read_csv('online_retail_cleaned_no_cancellations.csv')  # Load your dataset here (or pre-extracted unique stock codes)
    return model, scaler, label_encoder, data

# Load resources
rf_model, scaler, label_encoder, data = load_model_and_data()

# Extract unique StockCodes
unique_stock_codes = data['StockCode'].unique()  # Replace 'StockCode' with your column name
unique_stock_codes.sort()  # Sort for better usability

# Title of the web app
st.title('Unit Price Prediction for Stock Items')
st.markdown("""
This application predicts the unit price of a stock item based on various factors such as the stock code, day of the week, hour of the day, and month of the year.
Please provide the necessary details below to get a prediction.
""")

# Create a dropdown to select a Stock Code
stock_code = st.selectbox('Select Stock Code', unique_stock_codes, help="Choose a valid stock code from the dataset.")

# Create other input fields for the user
invoice_weekday = st.selectbox(
    'Select Invoice Weekday',
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    help="Choose the day of the week for the invoice."
)
invoice_hour = st.slider('Select Invoice Hour', 0, 23, 15, help="Pick the hour of the day (0-23). Default is 3 PM.")
invoice_month = st.slider('Select Invoice Month', 1, 12, 11, help="Pick the invoice month. Default is November.")

# Mapping weekdays to integers (0=Monday, 6=Sunday)
weekday_mapping = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3,
    'Friday': 4, 'Saturday': 5, 'Sunday': 6
}
invoice_weekday_int = weekday_mapping[invoice_weekday]

# When the user presses the button to predict
if st.button('Predict Unit Price'):
    # Encode the stock code
    try:
        stock_code_encoded = label_encoder.transform([stock_code])[0]
    except ValueError:
        st.error("Invalid Stock Code! Please try a valid one.")
        st.stop()

    # Create the input features (replace lagged values with actual user input if available)
    input_features = pd.DataFrame({
        'StockCode_Encoded': [stock_code_encoded],
        'Quantity': [0],  # Replace with actual input if available
        'InvoiceHour': [invoice_hour],
        'InvoiceWeekday': [invoice_weekday_int],
        'InvoiceMonth': [invoice_month],
        'Quantity_Lagged_Avg': [0],  # Replace with actual input if available
        'UnitPrice_Lagged_Avg': [0],  # Replace with actual input if available
        'Quantity_UnitPrice_Lagged_Avg': [0]  # Replace with actual input if available
    })

    # Scale the features
    scaled_features = scaler.transform(input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])
    input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']] = scaled_features

    # Predict the unit price
    try:
        predicted_price = rf_model.predict(input_features)
        st.write(f"Predicted Unit Price for StockCode {stock_code} on {invoice_weekday} at {invoice_hour}:00 is: £{predicted_price[0]:.2f}")
    except Exception as e:
        st.error(f"An error occurred during prediction: {str(e)}")

# Additional section for file upload (optional)
st.sidebar.title("Upload Model Files (Optional)")
st.sidebar.markdown("If the required files are not found, you can upload them here.")
uploaded_model = st.sidebar.file_uploader("Upload Random Forest Model (.pkl)", type=["pkl"])
uploaded_scaler = st.sidebar.file_uploader("Upload Scaler (.pkl)", type=["pkl"])
uploaded_encoder = st.sidebar.file_uploader("Upload Label Encoder (.pkl)", type=["pkl"])

# Replace files with uploaded versions (if provided)
if uploaded_model:
    rf_model = joblib.load(uploaded_model)
    st.sidebar.success("Random Forest model uploaded successfully!")
if uploaded_scaler:
    scaler = joblib.load(uploaded_scaler)
    st.sidebar.success("Scaler uploaded successfully!")
if uploaded_encoder:
    label_encoder = joblib.load(uploaded_encoder)
    st.sidebar.success("Label encoder uploaded successfully!")


In [1]:
import pandas as pd
data = pd.read_csv('online_retail_cleaned_no_cancellations.csv')
print(data.columns)


Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice(£)', 'CustomerID', 'Country', 'Sales(£)', 'InvoiceMonth',
       'InvoiceDay', 'InvoiceHour', 'InvoiceWeekday', 'is_canceled'],
      dtype='object')


Working APP

In [ ]:
import streamlit as st
import pandas as pd
import joblib
import os

# Function to load model, scaler, label encoder, and dataset
@st.cache_resource
def load_model_and_data():
    # Check if all required files exist
    if not os.path.exists('random_forest_model.pkl') or not os.path.exists('scaler.pkl') or not os.path.exists('label_encoder.pkl') or not os.path.exists('online_retail_cleaned_no_cancellations.csv'):
        st.error("Required files (model/scaler/encoder/data) are missing! Please check the directory.")
        st.stop()
    

    # Load the trained model, scaler, label encoder, and data
    model = joblib.load('random_forest_model.pkl')
    scaler = joblib.load('scaler.pkl')
    label_encoder = joblib.load('label_encoder.pkl')
    data = pd.read_csv('online_retail_cleaned_no_cancellations.csv')  # Load your dataset here (or pre-extracted unique stock codes)
    return model, scaler, label_encoder, data

# Load resources
rf_model, scaler, label_encoder, data = load_model_and_data()

# Extract unique StockCodes
unique_stock_codes = data['StockCode'].unique()  # Replace 'StockCode' with your column name
unique_stock_codes.sort()  # Sort for better usability

# Title of the web app
st.title('Unit Price Prediction for Stock Items')
st.markdown("""
This application predicts the unit price of a stock item based on various factors such as the stock code, day of the week, hour of the day, and month of the year.
Please provide the necessary details below to get a prediction.
""")

# Create a dropdown to select a Stock Code
stock_code = st.selectbox('Select Stock Code', unique_stock_codes, help="Choose a valid stock code from the dataset.")

# Create other input fields for the user
invoice_weekday = st.selectbox(
    'Select Invoice Weekday',
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    help="Choose the day of the week for the invoice."
)
invoice_hour = st.slider('Select Invoice Hour', 0, 23, 15, help="Pick the hour of the day (0-23). Default is 3 PM.")
invoice_month = st.slider('Select Invoice Month', 1, 12, 11, help="Pick the invoice month. Default is November.")

# Mapping weekdays to integers (0=Monday, 6=Sunday)
weekday_mapping = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3,
    'Friday': 4, 'Saturday': 5, 'Sunday': 6
}
invoice_weekday_int = weekday_mapping[invoice_weekday]

# When the user presses the button to predict
if st.button('Predict Unit Price'):
    # Encode the stock code
    try:
        stock_code_encoded = label_encoder.transform([stock_code])[0]
    except ValueError:
        st.error("Invalid Stock Code! Please try a valid one.")
        st.stop()

    # Create the input features with original feature names
    input_features = pd.DataFrame({
        'StockCode_Encoded': [stock_code_encoded],
        'Quantity': [0],  # Replace with actual input if available
        'Hour': [invoice_hour],  # Renamed to match training data
        'Day': [invoice_weekday_int],  # Renamed to match training data
        'Month': [invoice_month],  # Renamed to match training data
        'Quantity_Lagged_Avg': [0],  # Replace with actual input if available
        'UnitPrice_Lagged_Avg': [0],  # Replace with actual input if available
        'Quantity_UnitPrice_Lagged_Avg': [0]  # Replace with actual input if available
    })

    # Scale the features
    scaled_features = scaler.transform(input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']])
    input_features[['Quantity', 'Quantity_Lagged_Avg', 'UnitPrice_Lagged_Avg']] = scaled_features

    # Predict the unit price
    try:
        predicted_price = rf_model.predict(input_features)
        st.write(f"Predicted Unit Price for StockCode {stock_code} on {invoice_weekday} at {invoice_hour}:00 is: £{predicted_price[0]:.2f}")
    except Exception as e:
        st.error(f"An error occurred during prediction: {str(e)}")

